# Assignment 4

**Due to**: TBD

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Fact checking, Neural Languange Inference (**NLI**)

# Intro

This assignment is centred on a particular and emerging NLP task, formally known as **fact checking** (or fake checking). As AI techniques become more and more powerful, reaching amazing results, such as image and text generation, it is more than ever necessary to build tools able to distinguish what is real from what is fake.

Here we focus on a small portion of the whole fact checking problem, which aims to determine whether a given statement (fact) conveys a trustworthy information or not. 

More precisely, given a set of evidences and a fact to verify, we would like our model to correctly predict whether the fact is true or fake.

In particular, we will see:

*   Dataset preparation (analysis and pre-processing)
*   Problem formulation: multi-input binary classification
*   Defining an evaluation method
*   Simple sentence embedding
*   Neural building blocks
*   Neural architecture extension

# The FEVER dataset

First of all, we need to choose a dataset. In this assignment we will rely on the [FEVER dataset](https://fever.ai).

The dataset is about facts taken from Wikipedia documents that have to be verified. In particular, facts could face manual modifications in order to define fake information or to give different formulations of the same concept.

The dataset consists of 185,445 claims manually verified against the introductory sections of Wikipedia pages and classified as ```Supported```, ```Refuted``` or ```NotEnoughInfo```. For the first two classes, systems and annotators need to also return the combination of sentences forming the necessary evidence supporting or refuting the claim.

## Dataset structure

Relevant data is divided into two file types. Information concerning the fact to verify, its verdict and associated supporting/opposing statements are stored in **.jsonl** format. In particular, each JSON element is a python dictionary with the following relevant fields:

*    **ID**: ID associated to the fact to verify.

*    **Verifiable**: whether the fact has been verified or not: ```VERIFIABLE``` or ```NOT VERIFIABLE```.
    
*    **Label**: the final verdict on the fact to verify: ```SUPPORTS```, ```REFUTES``` or ```NOT ENOUGH INFO```.
    
*    **Claim**: the fact to verify.
    
*    **Evidence**: a nested list of document IDs along with the sentence ID that is associated to the fact to verify. In particular, each list element is a tuple of four elements: the first two are internal annotator IDs that can be safely ignored; the third term is the document ID (called URL) and the last one is the sentence number (ID) in the pointed document to consider.

**Some Examples**

---

**Verifiable**

{"id": 202314, "verifiable": "VERIFIABLE", "label": "REFUTES", "claim": "The New Jersey Turnpike has zero shoulders.", "evidence": [[[238335, 240393, "New_Jersey_Turnpike", 15]]]}

---

**Not Verifiable**

{"id": 113501, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Grease had bad reviews.", "evidence": [[[133128, null, null, null]]]}

---

## Some simplifications and pre-processing

We are only interested in verifiable facts. Thus, we can filter out all non-verifiable claims.

Additionally, the current dataset format does not contain all necessary information for our classification purposes. In particular, we need to download Wikipedia documents and replace reported evidence IDs with the corresponding text.

Don't worry about that! We are providing you the already pre-processed dataset so that you can concentrate on the classification pipeline (pre-processing, model definition, evaluation and training).

You can download the zip file containing all set splits (train, validation and test) of the FEVER dataset by clicking on this [link](https://drive.google.com/file/d/1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1/view?usp=sharing). Alternatively, run the below code cell to automatically download it on this notebook.

**Note**: each dataset split is in .csv format. Feel free to inspect the whole dataset!

In [1]:
import os
import requests
import zipfile

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    toy_data_path = os.path.join(data_path, 'fever_data.zip')
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(toy_url,
                                   params={'id': toy_data_url_id},
                                   stream=True)
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data('dataset')

Download completed!
Extracting dataset...
Extraction completed!


# Classification dataset

At this point, you should have a reay-to-go dataset! Note that the dataset format changed as well! In particular, we split the evidence set associated to each claim, in order to build (claim, evidence) pairs. The classification label is propagated as well.

We'll motivate this decision in the next section!

Just for clarity, here's an example of the pre-processed dataset:

---

**Claim**: "Wentworth Miller is yet to make his screenwriting debut."

**Evidence**: "2	He made his screenwriting debut with the 2013 thriller film Stoker .	Stoker	Stoker (film)"

**Label**: Refutes

---

**Note**: The dataset requires some text cleaning as you may notice!


# Problem formulation

As mentioned at the beginning of the assignment, we are going to formulate the fact checking problem as a binary classification task.

In particular, each dataset sample is comprised of:

*     A claim to verify
*     A set of semantically related statements (evidence set)
*     Fact checking label: either evidences support or refute the claim.

Handling the evidence set from the point of view of neural models may imply some additional complexity: if the evidence set is comprised of several sentences we might incur in memory problems.

To this end, we further simplify the problem by building (claim, evidence) pairs. The fact checking label is propagated as well.

Example:

     Claim: c1 
     Evidence set: [e1, e2, e3]
     Label: S (support)

--->

    (c1, e1, S),
    (c1, e2, S),
    (c1, e3, S)

## Schema

The overall binary classification problem is summed up by the following (simplified) schema

![](https://drive.google.com/uc?export=view&id=1Wm_YBnFwgJtxcWEBpPbTBEVkpKaL08Jp)

Don't worry too much about the **Encoding** block for now. We'll give you some simple guidelines about its definition. For the moment, stick to the binary classification task definition where, in this case, we have 2 inputs: the claim to verify and one of its associated evidences.

# Architecture Guidelines

There are many neural architectures that follow the above schema. To avoid phenomena like the writer's block, in this section we are going to give you some implementation guidelines.

In particular, we would like you to test some implementations so that you explore basic approaches (neural baselines) and use them as building blocks for possible extensions.

## Handling multiple inputs

The first thing to notice is that we are in a multi-input scenario. In particular, each sample is comprised of a fact and its asssociated evidence statement.

Each of these input is encoded as a sequence of tokens. In particular, we will have the following input matrices:

*    Claim: [batch_size, max_tokens]
*    Evidence: [batch_size, max_tokens]

Moreover, after the embedding layer, we'll have:

*    Claim: [batch_size, max_tokens, embedding_dim]
*    Evidence: [batch_size, max_tokens, embedding_dim]

But, we would like to have a 2D input to our classifier, since we have to give an answer at pair level. Therefore, for each sample, we would expect the following input shape to our classification block:

*   Classification input shape: [batch_size, dim]

**How to do that?**

We inherently need to reduce the token sequence to a single representation. This operation is formally known as **sentence embedding**. Indeed, we are trying to compress the information of a whole sequence into a single embedding vector.

Here are some simple solutions that we ask you to try out:

*   Encode token sequences via a RNN and take the last state as the sentence embedding.

*   Encode token sequences via a RNN and average all the output states.

*   Encode token sequences via a simple MLP layer. In particular, if your input is a [batch_size, max_tokens, embedding_dim] tensor, the matrix multiplication works on the **max_tokens** dimension, resulting in a [batch_size, embedding_dim] 2D matrix.

*   Compute the sentence embedding as the mean of its token embeddings (**bag of vectors**).

## Merging multi-inputs

At this point, we have to think about **how** we should merge evidence and claim sentence embeddings.

For simplicity, we stick to simple merging strategies:

*     **Concatenation**: define the classification input as the concatenation of evidence and claim sentence embeddings

*     **Sum**: define the classification input as the sum of evidence and claim sentence embeddings

*     **Mean**: define the classification input as the mean of evidence and claim sentence embeddings

For clarity, if we the sentence embedding of a single input has shape [batch_size, embedding_dim], then the classification input has shape:

*     **Concatenation**: [batch_size, 2 * embedding_dim]

*     **Sum**: [batch_size, embedding_dim]

*     **Mean**: [batch_size, embedding_dim]

# A simple extension

Lastly, we ask you to modify previously defined neural architectures by adding an additional feature to the classification input.

We would like to see if some similarity information between the claim to verify and one of its associated evidence might be useful to the classification.

Compute the cosine similarity metric between the two sentence embeddings and concatenate the result to the classification input.

For clarity, since the cosine similarity of two vectors outputs a scalar value, the classification input shape is modified as follows:

*     **Concatenation**: [batch_size, 2 * embedding_dim + 1]

*     **Sum**: [batch_size, embedding_dim + 1]

*     **Mean**: [batch_size, embedding_dim + 1]



# Performance evaluation

Due to our simplifications, obtained results are not directly compatible with a traditional fact checking method that considers the evidence set as a whole.

Thus, we need to consider two types of evaluations.

**Multi-input classification evaluation**

This type of evaluation is the easiest and concerns computing evaluation metrics, such as accuracy, f1-score, recall and precision, of our pre-processed dataset.

In other words, we assess the performance of chosen classifiers.

**Claim verification evaluation**

However, if we want to give an answer concerning the claim itself, we need to consider the whole evidence set. 

Intuitively, for a given claim, we consider all its corresponding (claim, evidence) pairs and their corresponding classification outputs. 

At this point, all we need to do is to compute the final predicted claim label via majority voting.

Example:

    Claim: c1
    Evidence set: e1, e2, e3
    True label: S

    Pair outputs:
    (c1, e1) -> S (supports)
    (c1, e2) -> S (supports)
    (c1, e3) -> R (refutes)

    Majority voting:
    S -> 2 votes
    R -> 1 vote

    Final label:
    c1 -> S

Lastly, we have to compute classification metrics just like before.

# Tips and Extras

## Extensions are welcome!

Is this task too easy for you? Are you curious to try out things you have seen during lectures (e.g. attention)? Feel free to try everything you want!

Don't forget to try neural baselines first!

## Comments and documentation

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

## Organization

We suggest you to divide your work into sections. This allows you to build clean and modular code, as well as easy to read and to debug.

A possible schema:

*   Dataset pre-processing
*   Dataset conversion
*   Model definition
*   Training
*   Evaluation
*   Comments/Summary

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

Don't forget that your feedback is very important! Your suggestions help us improving course material.

# FAQ

---

**Q: Can I do something text pre-processing?**

**A:** You have to! If you check text data, the majority of sentences need some cleaning.

---

**Q: I'm struggling with the implementation. Can you help me?**

**A:** Yes sure! Write us an email about your issue. If you are looking for a particular type of operation, you can easily check the documentation of the deep learning framework you are using (google is your friend).

---

**Q: Can I try other encoding strategies or neural architectures?**

**A:** Absolutely! Remember to try out recommended neural baselines first and only then proceed with your extensions.

---

## Dataset pre-processing

### Dataset loading and inspection

In [2]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("dataset/train_pairs.csv")
val_df   = pd.read_csv("dataset/val_pairs.csv")
test_df  = pd.read_csv("dataset/test_pairs.csv")

In [3]:
# inspect dataset
train_df.head()

,Unnamed: 0,Claim,Evidence,ID,Label
0,0,Chris Hemsworth appeared in A Perfect Getaway.,2\tHemsworth has also appeared in the science ...,3,SUPPORTS
1,1,Roald Dahl is a writer.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,7,SUPPORTS
2,2,Roald Dahl is a governor.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,8,REFUTES
3,3,Ireland has relatively low-lying mountains.,10\tThe island 's geography comprises relative...,9,SUPPORTS
4,4,Ireland does not have relatively low-lying mou...,10\tThe island 's geography comprises relative...,10,REFUTES


### Preprocessing

In [4]:
# examine evidences
print(train_df["Evidence"][0])
print(train_df["Evidence"][993])

2	Hemsworth has also appeared in the science fiction action film Star Trek -LRB- 2009 -RRB- , the thriller adventure A Perfect Getaway -LRB- 2009 -RRB- , the horror comedy The Cabin in the Woods -LRB- 2012 -RRB- , the dark-fantasy action film Snow White and the Huntsman -LRB- 2012 -RRB- , the war film Red Dawn -LRB- 2012 -RRB- , and the biographical sports drama film Rush -LRB- 2013 -RRB- .	Star Trek	Star Trek (film)	A Perfect Getaway	A Perfect Getaway	The Cabin in the Woods	The Cabin in the Woods	Snow White and the Huntsman	Snow White and the Huntsman	Red Dawn	Red Dawn (2012 film)	Rush	Rush (2013 film)
25	She has appeared in Time 100 most influential people in the world -LRB- 2010 and 2015 -RRB- , Forbes top-earning women in music -LRB- 2011 -- 2015 -RRB- , Forbes 100 most powerful women -LRB- 2015 -RRB- , and Forbes Celebrity 100 -LRB- 2016 -RRB- .	Time	Time (magazine)	100 most influential people in the world	Time 100	Forbes	Forbes	100 most powerful women	The World's 100 Most Powerfu

In [5]:
import re
import nltk
from nltk.corpus import stopwords

try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

def preprocess_text(evidence) :
  # remove leading tabs
  pattern = r'[0-9]+\t'
  evidence = re.sub(pattern, '', evidence)

  # remove pronunciations
  pattern = r'-LSB-.*-RSB-(\s;)*'
  evidence = re.sub(pattern, '', evidence)

  # convert -LRB- -RRB- to ( or )
  pattern = r'-LRB-'
  evidence = re.sub(pattern, '(', evidence)
  pattern = r'-RRB-'
  evidence = re.sub(pattern, ')', evidence)

  # fix double dashes (--)
  pattern = r'\-\-'
  evidence = re.sub(pattern, '-', evidence)

  # remove double spaces
  pattern = r'(\s)\s+'
  evidence = re.sub(pattern, r'\1', evidence)

  # remove trailing words (hyperlinks)
  pattern = r'.\t.*$'
  evidence = re.sub(pattern, '.', evidence)

  # make sure all possesive 's are split from other words
  pattern = r"(\s.+)'s"
  evidence = re.sub(pattern, r"\1 's", evidence)

  REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
  #evidence = re.sub(REPLACE_BY_SPACE_RE, ' ', evidence)

  return evidence

train_df["Claim"] = train_df["Claim"].apply(preprocess_text)
train_df["Evidence"] = train_df["Evidence"].apply(preprocess_text)
val_df["Claim"] = val_df["Claim"].apply(preprocess_text)
val_df["Evidence"] = val_df["Evidence"].apply(preprocess_text)
test_df["Claim"] = test_df["Claim"].apply(preprocess_text)
test_df["Evidence"] = test_df["Evidence"].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
print(train_df["Claim"][0])
print(train_df["Evidence"][0])

Chris Hemsworth appeared in A Perfect Getaway.
Hemsworth has also appeared in the science fiction action film Star Trek ( 2009 ) , the thriller adventure A Perfect Getaway ( 2009 ) , the horror comedy The Cabin in the Woods ( 2012 ) , the dark-fantasy action film Snow White and the Huntsman ( 2012 ) , the war film Red Dawn ( 2012 ) , and the biographical sports drama film Rush ( 2013 ) .


## Dataset conversion

In [7]:
def convert_facts(df) :
  subdf = df[["Claim", "Evidence", "Label"]]
  return [tuple(x) for x in subdf.to_numpy()]

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
import gensim.downloader as api
from nltk.tokenize.treebank import TreebankWordTokenizer

embedding_dimension = 50
glove = api.load(f"glove-wiki-gigaword-{embedding_dimension}")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [9]:
# tokenizer based on penn treebank
def tokenize_df(df) :
  word_tokenizer = TreebankWordTokenizer()
  tmp = pd.DataFrame(columns=["Claim", "Evidence", "Label"])
  for col in ["Claim", "Evidence"]:
    tmp[col] = df[col].apply(lambda r: word_tokenizer.tokenize(r.lower()))
    #tmp[col] = np.array([word_tokenizer.tokenize(r) for r in tmp[col]])
  tmp["Label"] = df["Label"]
  return tmp

In [10]:
tok_train_df = tokenize_df(train_df)
tok_train_df.head()

,Claim,Evidence,Label
0,"[chris, hemsworth, appeared, in, a, perfect, g...","[hemsworth, has, also, appeared, in, the, scie...",SUPPORTS
1,"[roald, dahl, is, a, writer, .]","[roald, dahl, (, 13, september, 1916, -, 23, n...",SUPPORTS
2,"[roald, dahl, is, a, governor, .]","[roald, dahl, (, 13, september, 1916, -, 23, n...",REFUTES
3,"[ireland, has, relatively, low-lying, mountain...","[the, island, 's, geography, comprises, relati...",SUPPORTS
4,"[ireland, does, not, have, relatively, low-lyi...","[the, island, 's, geography, comprises, relati...",REFUTES


In [11]:
# build vocab
def build_vocab_from_df(df) :
  ret = []
  for col in ["Claim", "Evidence"] :
    for r in df[col] :
      ret += r
  ret = list(dict.fromkeys(ret))
  return ret

vocab = list(glove.vocab.keys())               # glove vocab

vocab_v1 = build_vocab_from_df(tok_train_df)   # add unique terms from claims
oov_v1 = [w for w in vocab_v1 if w not in vocab]
vocab += oov_v1 # remove duplicates between glove vocab and dataset vocab while preserving order

vocab_v2 = build_vocab_from_df(tokenize_df(val_df))
oov_v2 = [w for w in vocab_v2 if w not in vocab]
vocab += oov_v2

vocab_v3 = build_vocab_from_df(tokenize_df(test_df))
oov_v3 = [w for w in vocab_v3 if w not in vocab]
vocab += oov_v3

#vocab = list(dict.fromkeys(vocab))
print(f"vocab len: {len(vocab)}")

vocab len: 404454


In [12]:
# build word -> int encoding 
word_to_idx = dict(zip(vocab, range(1, len(vocab)+1))) # start from 1 to reserve 0 to padding

In [13]:
def encode_sent(sent, word_to_idx=word_to_idx) :
  return [word_to_idx[w] for w in sent]

def encode_df(tok_df, word_to_idx) :
  enc_df = pd.DataFrame(columns=["Claim", "Evidence", "Label"])
  for col in ["Claim", "Evidence"] :
    enc_df[col] = tok_df[col].apply(lambda s: encode_sent(s, word_to_idx))
  enc_df["Label"] = tok_df["Label"].apply(lambda x: 1 if x=="SUPPORTS" else 0)
  return enc_df

In [14]:
# encode dataset
enc_train_df = encode_df(tok_train_df, word_to_idx)

In [15]:
enc_train_df.head()

,Claim,Evidence,Label
0,"[2103, 107954, 790, 7, 8, 2616, 20647, 3]","[107954, 32, 53, 790, 7, 1, 1122, 3955, 609, 3...",1
1,"[53403, 21758, 15, 8, 1542, 3]","[53403, 21758, 24, 677, 442, 6907, 12, 1022, 4...",1
2,"[53403, 21758, 15, 8, 1005, 3]","[53403, 21758, 24, 677, 442, 6907, 12, 1022, 4...",0
3,"[1323, 32, 2224, 46585, 2755, 3]","[1, 584, 10, 4214, 7817, 2224, 46585, 2755, 27...",1
4,"[1323, 261, 37, 34, 2224, 46585, 2755, 3]","[1, 584, 10, 4214, 7817, 2224, 46585, 2755, 27...",0


In [16]:
def build_embedding_matrix(vocab=vocab, embedding_model=glove, embedding_dimension=50) :
  matrix = [np.zeros((embedding_dimension))] # first element reserved to padding and set to all zeros
  for w in vocab :
    if w in embedding_model.vocab :
      matrix.append(embedding_model[w])
    else:
      matrix.append(np.random.uniform(low=-1.0, high=1.0, size=embedding_dimension))
  return np.array(matrix)

In [17]:
embedding_matrix = build_embedding_matrix(vocab, glove, 50)
embedding_matrix.shape

(404455, 50)

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, Bidirectional, \
                                    LSTM, GRU, SimpleRNN, Lambda, TimeDistributed, \
                                    Masking, GlobalMaxPool1D, GlobalAveragePooling1D, \
                                    Activation, Concatenate, Flatten, Reshape, SpatialDropout1D, Dropout

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity

tf.random.set_seed(42)

In [19]:
embedding_layer = Embedding(
    embedding_matrix.shape[0],    # vocab size 
    embedding_matrix.shape[1],    # embedding dimension
    embeddings_initializer=tf.initializers.Constant(embedding_matrix),
    mask_zero = True,
    name = 'embedding',
    trainable = False
)

In [20]:
max_length = pad_sequences(enc_train_df['Evidence'], padding='post').shape[1]

In [31]:
sent_emb = Sequential([
  #Input((max_length,embedding_dimension,)),
  Bidirectional(LSTM(64, return_sequences=False)),
  #Masking(mask_value=0.0),
  #Lambda(lambda x: tf.reduce_max(x, axis=1)),
  #GlobalAveragePooling1D(),
  #GlobalMaxPool1D(),
  Dense(64),
  Dense(embedding_dimension),
], name="sentence_embedding")

In [22]:
enc_val_df = encode_df(tokenize_df(val_df), word_to_idx)
enc_test_df = encode_df(tokenize_df(test_df), word_to_idx)

In [23]:
padded_train_claim = pad_sequences(enc_train_df['Claim'], maxlen=max_length, padding='post')
padded_train_evidence = pad_sequences(enc_train_df['Evidence'], maxlen=max_length, padding='post')

padded_val_claim = pad_sequences(enc_val_df['Claim'], maxlen=max_length, padding='post')
padded_val_evidence = pad_sequences(enc_val_df['Evidence'], maxlen=max_length, padding='post')

padded_test_claim = pad_sequences(enc_test_df['Claim'], maxlen=max_length, padding='post')
padded_test_evidence = pad_sequences(enc_test_df['Evidence'], maxlen=max_length, padding='post')

## Model definition

In [32]:
tf.keras.backend.clear_session()

def build_model(max_length) :

  claim = Input((max_length,), name='claim')
  evidence = Input((max_length), name='evidence')

  embedding_c = embedding_layer(claim)
  embedding_e = embedding_layer(evidence)

  sent_emb_c = sent_emb(embedding_c)
  sent_emb_e = sent_emb(embedding_e)

  output = Concatenate(name='refined_input')([sent_emb_c, sent_emb_e])
  
  #output = Reshape((2*embedding_dimension,1))(output)
  #output = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(output)
  #output = TimeDistributed(Dense(32, activation='relu'))(output)
  #output = Flatten()(output)
  output = Dense(64, activation='relu')(output)
  output = Dense(32, activation='relu')(output)
  output = Dropout(0.5)(output)
  #output = Flatten()(output)
  output = Dense(1, activation='sigmoid')(output)

  return Model([claim, evidence], output)


model = build_model(max_length)

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
claim (InputLayer)              [(None, 144)]        0                                            
__________________________________________________________________________________________________
evidence (InputLayer)           [(None, 144)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 144, 50)      20222750    claim[0][0]                      
                                                                 evidence[0][0]                   
__________________________________________________________________________________________________
sentence_embedding (Sequential) (None, 50)           70386       embedding[6][0]       

In [ ]:
model.fit(x=(padded_train_claim, padded_train_evidence), 
          y=enc_train_df['Label'],
          batch_size=32,
          #steps_per_epoch=1000,
          epochs=20,
          validation_data=((padded_val_claim,padded_val_evidence), enc_val_df['Label'])
          )

Epoch 1/20
3805/3805 [==============================] - 96s 25ms/step - loss: 0.4641 - acc: 0.8010 - val_loss: 0.5968 - val_acc: 0.6775
Epoch 2/20
3805/3805 [==============================] - 91s 24ms/step - loss: 0.3972 - acc: 0.8364 - val_loss: 0.6275 - val_acc: 0.6955
Epoch 3/20
3660/3805 [===========================>..] - ETA: 3s - loss: 0.3712 - acc: 0.8454

In [ ]:
X1 = padded_train_claim[2].reshape(1, -1)
X2 = padded_train_evidence[2].reshape(1, -1)

In [ ]:
model((X1,X2))

In [ ]:
sent_emb(embedding_layer(X1))

## Training

## Evaluation

## Debug

In [ ]:
# Example 4
id1 = 0
#id1 = 78061
#id1 = 7612
v1 = sent_emb(pad_sequences(enc_train_df['Claim'], maxlen=max_length, padding='post')[id1].reshape(1, -1))
v2 = sent_emb(pad_sequences(enc_train_df['Evidence'], maxlen=max_length, padding='post')[id1].reshape(1, -1))
sim = cosine_similarity(v1, v2)
print(f'similarity between \"{train_df["Claim"][id1]}\" and \"{train_df["Evidence"][id1]}\":')
print(sim)

In [ ]:
# Example 1
id1 = 7612
id2 = 78061
v1 = sent_emb(pad_sequences(enc_train_df['Claim'], padding='post')[id1].reshape(1, -1))
v2 = sent_emb(pad_sequences(enc_train_df['Claim'], padding='post')[id2].reshape(1, -1))
sim = cosine_similarity(v1, v2)
print(f'similarity between \"{train_df["Claim"][id1]}\" and \"{train_df["Claim"][id2]}\":')
print(sim)

In [ ]:
# Example 2
id1 = 1
id2 = 2
v1 = sent_emb(pad_sequences(enc_train_df['Claim'], padding='post')[id1].reshape(1, -1))
v2 = sent_emb(pad_sequences(enc_train_df['Claim'], padding='post')[id2].reshape(1, -1))
sim = cosine_similarity(v1, v2)
print(f'similarity between \"{train_df["Claim"][id1]}\" and \"{train_df["Claim"][id2]}\":')
print(sim)
#embedding_layer(pad_sequences(enc_train_df['Claim'], padding='post')[0].reshape(1, -1)).shape

In [ ]:
# Example 3
id1 = 0
id2 = 3
v1 = sent_emb(pad_sequences(enc_train_df['Claim'], padding='post')[id1].reshape(1, -1))
v2 = sent_emb(pad_sequences(enc_train_df['Claim'], padding='post')[id2].reshape(1, -1))
sim = cosine_similarity(v1, v2)
print(f'similarity between \"{train_df["Claim"][id1]}\" and \"{train_df["Claim"][id2]}\":')
print(sim)
#embedding_layer(pad_sequences(enc_train_df['Claim'], padding='post')[0].reshape(1, -1)).shape

In [ ]:
glove.similarity('american', 'swedish')

In [ ]:
len(enc_train_df["Evidence"][3])

In [ ]:
# debug
train_df[train_df["Evidence"].str.contains("Swift")]